In [1]:
# Import essential libraries
import tensorflow as tf
import numpy as np
import json
import pickle
import sys
import tensorflow.keras as keras
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
import os
gpus = tf.config.experimental.list_physical_devices(device_type="GPU")

gpu = len(tf.config.list_physical_devices('GPU')) > 0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is available


In [2]:
tf.__version__

'2.13.1'

In [2]:
# Load Data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
dataset = (tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(64)).batch(32)

In [3]:
# Ideas for inserting errors:
# Manipulate Class so that it results in error
# Cause Exploding / Vanishing Gradients.
# Look at adverserial attacks
# Look at Dropout

In [3]:
# 2-Layer MlP with relu activations
model = keras.Sequential()
model.add(keras.Input(shape=(28,28,)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation="sigmoid"))
model.add(keras.layers.Dense(50, activation="sigmoid"))
model.add(keras.layers.Dense(10))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 10)                510       
                                                                 
Total params: 84,060
Trainable params: 84,060
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Apply loss function and optimizer, then compile the model
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss_fn, metrics=["acc"])

In [5]:
# Train the model on dataset for given epochs
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 6s 2ms/step - loss: 0.6532 - acc: 0.8341 - val_loss: 0.3931 - val_acc: 0.8872
Epoch 2/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3764 - acc: 0.8875 - val_loss: 0.3355 - val_acc: 0.8950
Epoch 3/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3232 - acc: 0.9024 - val_loss: 0.3014 - val_acc: 0.9045
Epoch 4/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3004 - acc: 0.9085 - val_loss: 0.2879 - val_acc: 0.9137
Epoch 5/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2925 - acc: 0.9108 - val_loss: 0.2955 - val_acc: 0.9088
Epoch 6/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2792 - acc: 0.9137 - val_loss: 0.2750 - val_acc: 0.9168
Epoch 7/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2582 - acc: 0.9208 - val_loss: 0.2587 - val_acc: 0.9232
Epoch 8/10
1875/1875 [==============================] - 3s 1ms

In [7]:
# Save the model at the correct path
file_name = "two_layer_mlp.keras"
model.save('saved_models/' + file_name)

In [176]:
name = 'two_layer_mlp'
create_precalculations(name)


[Errno 17] File exists: 'saved_precalculations/two_layer_mlp'
Starting with calculations
Currently at batch: 0
Currently at batch: 1
Currently at batch: 2
Currently at batch: 3
Currently at batch: 4
Currently at batch: 5
Currently at batch: 6
Currently at batch: 7
Currently at batch: 8
Currently at batch: 9
Currently at batch: 10
Currently at batch: 11
Currently at batch: 12
Currently at batch: 13
Currently at batch: 14
Currently at batch: 15
Currently at batch: 16
Currently at batch: 17
Done with calculations. Saving to file.
Starting with calculations
Currently at batch: 0
Currently at batch: 1
Currently at batch: 2
Currently at batch: 3
Currently at batch: 4
Currently at batch: 5
Currently at batch: 6
Currently at batch: 7
Currently at batch: 8
Currently at batch: 9
Currently at batch: 10
Currently at batch: 11
Currently at batch: 12
Currently at batch: 13
Currently at batch: 14
Currently at batch: 15
Currently at batch: 16
Currently at batch: 17
Done with calculations. Saving to fi

In [175]:
# Add functions that need to be recomputed after every new model (precalculations)
# Add automatic creation of folder
def create_precalculations(model_name):
    model_name = 'two_layer_mlp'
    model = keras.models.load_model('saved_models/' + model_name + '.keras')

    # Create Folder for precalculations, if it doesn't exist yet.
    save_file_path = 'saved_precalculations/' + model_name
    try:
        os.mkdir(save_file_path)
    except OSError as error:
        print(error)

    create_subset(model, 'saved_precalculations/' + model_name + '/subset_activations_1.pickle')
    create_subset(model, 'saved_precalculations/' + model_name + '/subset_activations_2.pickle')
    create_subset(model, 'saved_precalculations/' + model_name + '/subset_activations_3.pickle')

In [168]:
def create_subset(model, file_path=None):
    dataset = (tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(64)).batch(32)
    partial_output_models = []

    dataset_size = sum(1 for _ in dataset)
    one_percent_size = dataset_size // 100

    # Take the first 1% of the dataset
    one_percent_dataset = dataset.take(one_percent_size)

    functions = []
    results = []
    for i in range(len(model.layers)):
        functions.append(K.function([model.layers[i].input],[model.layers[i].output]))

    print("Starting with calculations")
    layer_outputs = []
    for i, (batch, y) in enumerate(one_percent_dataset):
        print("Currently at batch: " + str(i))

        output = functions[0](batch)
        if i == 0:
            results.append(np.transpose(np.squeeze(output, axis=0)))
        else:
            results[0] = np.concatenate([results[0], np.transpose(np.squeeze(output, axis=0))], axis=1)

        for j in range(1, len(functions)):
            tmp = functions[j](output)
            output = tmp
            if i == 0:
                results.append(np.transpose(np.squeeze(output, axis=0)))
            else:
                results[j] = np.concatenate([results[j], np.transpose(np.squeeze(output, axis=0))], axis=1)

    print("Done with calculations. Saving to file.")

    with open(file_path, 'wb') as handle:
        pickle.dump(layer_outputs, handle, protocol=pickle.HIGHEST_PROTOCOL)